# **Kaggle – DataTops®**
Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes
- Última submission:
    - Mañana: 17 de febrero a las 5pm
    - Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica:
El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## 1. Librerías

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import urllib.request

## 2. Datos

In [2]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle
df = pd.read_csv("./data/train.csv", index_col= 0)

### 2.1 Exploración de los datos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [4]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [5]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [6]:
df.describe()

,Inches,Price_in_euros
count,912.000000,912.000000
mean,14.981579,1111.724090
std,1.436719,687.959172
min,10.100000,174.000000
25%,14.000000,589.000000
50%,15.600000,978.000000
75%,15.600000,1483.942500
max,18.400000,6099.000000


In [7]:
# Check cardinality of columns
for col in df.columns:
    print(f"Column {col} has: {df[col].nunique()} unique values.")

Column Company has: 19 unique values.
Column Product has: 480 unique values.
Column TypeName has: 6 unique values.
Column Inches has: 17 unique values.
Column ScreenResolution has: 36 unique values.
Column Cpu has: 107 unique values.
Column Ram has: 9 unique values.
Column Memory has: 37 unique values.
Column Gpu has: 93 unique values.
Column OpSys has: 9 unique values.
Column Weight has: 165 unique values.
Column Price_in_euros has: 603 unique values.


### 2.3 Definir X e y

In [8]:
X = df.drop(['Price_in_euros'], axis=1)
y = df['Price_in_euros'].copy()
X.shape

(912, 11)

In [9]:
y.shape

(912,)

### 2.4 Dividir X_train, X_test, y_train, y_test

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [11]:
X_train

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
laptop_ID,,,,,,,,,,,
1118,HP,ZBook 17,Workstation,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,AMD FirePro W6150M,Windows 7,3.0kg
153,Dell,Inspiron 5577,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1050,Windows 10,2.56kg
275,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.9GHz,8GB,512GB SSD,Intel Iris Graphics 550,macOS,1.37kg
1100,HP,EliteBook 840,Notebook,14.0,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,500GB HDD,Intel HD Graphics 520,Windows 7,1.54kg
131,Dell,Inspiron 5770,Notebook,17.3,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,16GB,256GB SSD + 2TB HDD,AMD Radeon 530,Windows 10,2.8kg
...,...,...,...,...,...,...,...,...,...,...,...
578,HP,14-am079na (N3710/8GB/2TB/W10),Notebook,14.0,1366x768,Intel Pentium Quad Core N3710 1.6GHz,8GB,2TB HDD,Intel HD Graphics 405,Windows 10,1.94kg
996,Lenovo,IdeaPad 320-15ABR,Notebook,15.6,Full HD 1920x1080,AMD A12-Series 9720P 3.6GHz,6GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg
770,Dell,Latitude 7280,Ultrabook,12.5,Full HD 1920x1080,Intel Core i7 7600U 2.8GHz,16GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.18kg


In [12]:
y_train

laptop_ID
1118    2899.00
153     1249.26
275     1958.90
1100    1030.99
131     1396.00
         ...   
578      389.00
996      549.00
770     1859.00
407      306.00
418     1943.00
Name: Price_in_euros, Length: 729, dtype: float64

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

In [13]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 729 entries, 1118 to 418
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           729 non-null    object 
 1   Product           729 non-null    object 
 2   TypeName          729 non-null    object 
 3   Inches            729 non-null    float64
 4   ScreenResolution  729 non-null    object 
 5   Cpu               729 non-null    object 
 6   Ram               729 non-null    object 
 7   Memory            729 non-null    object 
 8   Gpu               729 non-null    object 
 9   OpSys             729 non-null    object 
 10  Weight            729 non-null    object 
dtypes: float64(1), object(10)
memory usage: 68.3+ KB


In [14]:
# Drop "Product" column
X_train = X_train.drop(columns="Product")

In [15]:
X_test = X_test.drop(columns="Product")

In [16]:
# get dummies with some categorical columns
X_train = pd.get_dummies(X_train, columns = ["Company","TypeName","OpSys"], drop_first=True, dtype=int)
X_test = pd.get_dummies(X_test, columns = ["Company","TypeName","OpSys"], drop_first=True, dtype=int)
X_test = X_test.reindex(columns=X_train.columns,fill_value=0)

In [17]:
# Convert Ram into numeric
X_train["Ram"] = (X_train["Ram"].str.replace("GB","",regex = False).astype(int))

X_test["Ram"] = (X_test["Ram"].str.replace("GB","",regex = False).astype(int))

In [18]:
# Get total pixel count
res_train = X_train["ScreenResolution"].str.extract(r"(\d+)x(\d+)")
res_test = X_test["ScreenResolution"].str.extract(r"(\d+)x(\d+)")

X_train["res_width"] = res_train[0].astype(int)
X_train["res_height"] = res_train[1].astype(int)

X_test["res_width"] = res_test[0].astype(int)
X_test["res_height"] = res_test[1].astype(int)


X_train["pixel_count"] = X_train["res_width"] * X_train["res_height"]
X_test["pixel_count"] = X_test["res_width"] * X_test["res_height"]


In [19]:
X_train = X_train.drop(columns=["ScreenResolution", "res_width", "res_height"])
X_test = X_test.drop(columns=["ScreenResolution", "res_width", "res_height"])


In [20]:
# convert memory into total storage
import re

def extract_storage(mem): # create function to extract numeric values and sum
    sizes = re.findall(r"(\d+)(TB|GB)", mem)
    total = 0
    for size, unit in sizes:
        size = int(size)
        if unit == "TB":
            total += size * 1024
        else:
            total += size
    return total

# create new column using the function to get the information
X_train["storage_gb"] = X_train["Memory"].apply(extract_storage)
X_test["storage_gb"] = X_test["Memory"].apply(extract_storage)


In [21]:
# drop memory column
X_train = X_train.drop(columns=["Memory"])
X_test = X_test.drop(columns=["Memory"])

In [22]:
# extract cpu brand
X_train["cpu_brand"] = X_train["Cpu"].str.extract(
    r"(Intel|AMD)", expand=False
).str.lower()

X_test["cpu_brand"] = X_test["Cpu"].str.extract(
    r"(Intel|AMD)", expand=False
).str.lower()

In [23]:
X_train = X_train.drop(columns=["Cpu"])
X_test = X_test.drop(columns=["Cpu"])

In [24]:
X_train = pd.get_dummies(X_train, columns = ["cpu_brand"], drop_first=True, dtype=int)
X_test = pd.get_dummies(X_test, columns = ["cpu_brand"], drop_first=True, dtype=int)
X_test = X_test.reindex(columns=X_train.columns,fill_value=0)

In [25]:
X_train.head()

,Inches,Ram,Gpu,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,...,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand_intel
laptop_ID,,,,,,,,,,,,,,,,,,,,,
1118,17.3,8,AMD FirePro W6150M,3.0kg,0,0,0,0,0,0,...,0,0,0,0,0,1,0,2073600,1024,1
153,15.6,16,Nvidia GeForce GTX 1050,2.56kg,0,0,0,1,0,0,...,0,0,0,1,0,0,0,2073600,512,1
275,13.3,8,Intel Iris Graphics 550,1.37kg,1,0,0,0,0,0,...,0,0,0,0,0,0,1,4096000,512,1
1100,14.0,4,Intel HD Graphics 520,1.54kg,0,0,0,0,0,0,...,0,0,0,0,0,1,0,2073600,500,1
131,17.3,16,AMD Radeon 530,2.8kg,0,0,0,1,0,0,...,0,0,0,1,0,0,0,2073600,2304,1


In [26]:
X_train["gpu_brand"] = "other"
X_test["gpu_brand"] = "other"


X_train.loc[X_train["Gpu"].str.contains("intel", case=False, na=False), "gpu_brand"] = "intel"
X_test.loc[X_test["Gpu"].str.contains("intel", case=False, na=False), "gpu_brand"] = "intel"

X_train.loc[X_train["Gpu"].str.contains("nvidia", case=False, na=False), "gpu_brand"] = "nvidia"
X_test.loc[X_test["Gpu"].str.contains("nvidia", case=False, na=False), "gpu_brand"] = "nvidia"

X_train.loc[X_train["Gpu"].str.contains("amd|radeon", case=False, na=False), "gpu_brand"] = "amd"
X_test.loc[X_test["Gpu"].str.contains("amd|radeon", case=False, na=False), "gpu_brand"] = "amd"


In [27]:
X_train = pd.get_dummies(X_train, columns = ["gpu_brand"], drop_first=True, dtype=int)
X_test = pd.get_dummies(X_test, columns = ["gpu_brand"], drop_first=True, dtype=int)
X_test = X_test.reindex(columns=X_train.columns,fill_value=0)

In [28]:
X_train.head()

,Inches,Ram,Gpu,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,...,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand_intel,gpu_brand_intel,gpu_brand_nvidia
laptop_ID,,,,,,,,,,,,,,,,,,,,,
1118,17.3,8,AMD FirePro W6150M,3.0kg,0,0,0,0,0,0,...,0,0,0,1,0,2073600,1024,1,0,0
153,15.6,16,Nvidia GeForce GTX 1050,2.56kg,0,0,0,1,0,0,...,0,1,0,0,0,2073600,512,1,0,1
275,13.3,8,Intel Iris Graphics 550,1.37kg,1,0,0,0,0,0,...,0,0,0,0,1,4096000,512,1,1,0
1100,14.0,4,Intel HD Graphics 520,1.54kg,0,0,0,0,0,0,...,0,0,0,1,0,2073600,500,1,1,0
131,17.3,16,AMD Radeon 530,2.8kg,0,0,0,1,0,0,...,0,1,0,0,0,2073600,2304,1,0,0


In [29]:
X_train = X_train.drop(columns=["Gpu"])
X_test = X_test.drop(columns=["Gpu"])

In [30]:
X_train.head()

,Inches,Ram,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,...,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand_intel,gpu_brand_intel,gpu_brand_nvidia
laptop_ID,,,,,,,,,,,,,,,,,,,,,
1118,17.3,8,3.0kg,0,0,0,0,0,0,1,...,0,0,0,1,0,2073600,1024,1,0,0
153,15.6,16,2.56kg,0,0,0,1,0,0,0,...,0,1,0,0,0,2073600,512,1,0,1
275,13.3,8,1.37kg,1,0,0,0,0,0,0,...,0,0,0,0,1,4096000,512,1,1,0
1100,14.0,4,1.54kg,0,0,0,0,0,0,1,...,0,0,0,1,0,2073600,500,1,1,0
131,17.3,16,2.8kg,0,0,0,1,0,0,0,...,0,1,0,0,0,2073600,2304,1,0,0


In [31]:
X_train["Weight"] = (X_train["Weight"].str.replace("kg", "", regex=False).astype(float))
X_test["Weight"] = (X_test["Weight"].str.replace("kg", "", regex=False).astype(float))



In [32]:
numeric_columns = [
    "Inches",
    "Ram",
    "pixel_count",
    "storage_gb",
    "Weight"
]


In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train[numeric_columns] = scaler.fit_transform(
    X_train[numeric_columns]
)

X_test[numeric_columns] = scaler.transform(
    X_test[numeric_columns]
)


In [34]:
X_test.head()

,Inches,Ram,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,...,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand_intel,gpu_brand_intel,gpu_brand_nvidia
laptop_ID,,,,,,,,,,,,,,,,,,,,,
451,0.41823,-0.046800,0.249005,0,0,0,0,0,0,1,...,0,1,0,0,0,-0.053975,1.147287,1,0,1
802,0.41823,-0.046800,0.069270,0,0,0,0,0,0,0,...,0,1,0,0,0,-0.053975,-0.735734,1,0,0
723,-0.70243,-0.046800,-0.245266,0,0,0,0,0,0,0,...,0,1,0,0,0,-0.053975,-0.735734,1,0,1
682,-0.70243,-0.849549,-0.604736,0,0,0,0,0,0,1,...,0,1,0,0,0,-0.053975,-1.004737,1,0,1
1306,0.41823,-0.849549,0.249005,0,1,0,0,0,0,0,...,0,1,0,0,0,-0.804610,-0.222947,1,1,0


In [35]:
X_train.shape, X_test.shape


((729, 37), (183, 37))

-----------------------------------------------------------------------------------------------------------------

## 4. Modelado

### 4.1 Baseline de modelos


In [36]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


ridge = Ridge(random_state=42)
rf = RandomForestRegressor(random_state=42)
gb_reg = GradientBoostingRegressor(random_state=42)
xgb_reg = XGBRegressor(
    random_state=42,
    n_jobs=-1)
lgbm_reg = LGBMRegressor(random_state=42)


In [37]:
ridge.fit(X_train,y_train)
y_test_pred = ridge.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
rmse

np.float64(397.066592171052)

In [38]:
rf.fit(X_train,y_train)
y_test_pred = rf.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
rmse

np.float64(392.3832456912601)

In [39]:
gb_reg.fit(X_train,y_train)
y_test_pred = gb_reg.predict(X_test)

rmse = root_mean_squared_error(y_test_pred,y_test)
rmse

np.float64(390.58411369352683)

In [40]:
xgb_reg.fit(X_train,y_train)
y_test_pred = xgb_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
rmse

np.float64(393.56698603897655)

In [41]:
lgbm_reg.fit(X_train,y_train)
y_test_pred = lgbm_reg.predict(X_test)

rmse = root_mean_squared_error(y_test, y_test_pred)
rmse

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000160 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 168
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 21
[LightGBM] [Info] Start training from score 1103.789314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

np.float64(353.40624382283397)

In [42]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
    lgbm_reg,
    X_test,
    y_test,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

scores


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000164 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 146, number of used features: 14
[LightGBM] [Info] Start training from score 1123.306849
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

array([-521.44961043, -319.29810252, -527.35904629, -491.23978266,
       -335.07614036])

In [119]:
lgbm_params = {
    "n_estimators": [300, 600],
    "learning_rate": [0.05, 0.1],
    "num_leaves": [31, 63],
    "max_depth": [-1, 10],
    "min_child_samples": [20, 40],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0]
}


In [121]:
from sklearn.model_selection import GridSearchCV

grid_lgbm = GridSearchCV(
    estimator=lgbm_reg,
    param_grid=lgbm_params,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_lgbm.fit(X_train,y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


c:\Users\nicho\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\ma\core.py:2896: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000977 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 168
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 21
[LightGBM] [Info] Start training from score 1103.789314
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=42), n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.05, 0.1], 'max_depth': [-1, 10],
                         'min_child_samples': [20, 40],
                         'n_estimators': [300, 600], 'num_leaves': [31, 63],
                         'subsample': [0.8, 1.0]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [123]:
grid_lgbm.best_estimator_

LGBMRegressor(learning_rate=0.05, max_depth=10, n_estimators=300,
              random_state=42, subsample=0.8)

In [124]:
grid_lgbm.best_score_

np.float64(-330.18211367850216)

In [125]:
y_test_pred = grid_lgbm.predict(X_test)
rmse = root_mean_squared_error(y_test,y_test_pred)
rmse

np.float64(346.83928318683456)

In [43]:
xgb_params = {
    "n_estimators": [200, 400],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 2]
}

In [ ]:


xgb_grid = GridSearchCV(
    estimator=xgb_reg,
    param_grid=xgb_params,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

xgb_grid.fit(X_train, y_train)

c:\Users\nicho\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\ma\core.py:2896: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=-1, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.05, 0.1], 'max_depth': [3, 5],
                         'n_estimators': [200, 400], 'reg_alpha': [0, 0.1],
                         'reg_lambda': [1, 2], 'subsample': [0.8, 1.0]},
             scoring='neg_root_mean_squared_error')

In [45]:
xgb_best = xgb_grid.best_estimator_
xgb_best

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=-1, num_parallel_tree=None, ...)

In [46]:
xgb_grid.best_params_

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 400,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'subsample': 1.0}

In [47]:
xgb_grid.best_score_

np.float64(-300.80096011259684)

In [48]:
y_test_pred = xgb_best.predict(X_test)
rmse = root_mean_squared_error(y_test, y_test_pred)
rmse

np.float64(325.5131601714949)

In [49]:
rf_params = {
    "n_estimators": [300, 500],
    "max_depth": [None, 20],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
    "max_features": ["sqrt"]
}


In [50]:
grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=rf_params,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1,
    verbose=2
)


In [51]:
grid_rf.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 20], 'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [300, 500]},
             scoring='neg_root_mean_squared_error', verbose=2)

In [52]:
rf_best = grid_rf.best_estimator_
rf_best

RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=500,
                      random_state=42)

In [53]:
grid_rf.best_params_

{'max_depth': 20,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

In [54]:
grid_rf.best_score_

np.float64(-302.5600529960351)

In [55]:
y_test_pred = grid_rf.predict(X_test)
rmse = root_mean_squared_error(y_test,y_test_pred)
rmse

np.float64(323.73463856809167)

### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

### 4.3 Optimización (up to you 🫰🏻)

In [56]:
new_df = df.copy()
new_df

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
laptop_ID,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00
...,...,...,...,...,...,...,...,...,...,...,...,...
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [57]:
X_pred = pd.read_csv("./data/test.csv", index_col= 0)
X_pred.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
laptop_ID,,,,,,,,,,,
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


In [58]:
X_pred.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
laptop_ID,,,,,,,,,,,
820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg
421,Asus,ZenBook Flip,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.27kg


In [59]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   Inches            391 non-null    float64
 4   ScreenResolution  391 non-null    object 
 5   Cpu               391 non-null    object 
 6   Ram               391 non-null    object 
 7   Memory            391 non-null    object 
 8   Gpu               391 non-null    object 
 9   OpSys             391 non-null    object 
 10  Weight            391 non-null    object 
dtypes: float64(1), object(10)
memory usage: 36.7+ KB


In [60]:
X_pred.columns

Index(['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight'],
      dtype='object')

 ## 2. Replicar el procesado para ``test.csv``

In [61]:
# Drop "Product" column
X_pred = X_pred.drop(columns="Product")

In [62]:
X_pred = pd.get_dummies(X_pred, columns = ["Company","TypeName","OpSys"], drop_first=True, dtype=int)

In [63]:
# Convert Ram into numeric
X_pred["Ram"] = (X_pred["Ram"].str.replace("GB","",regex = False).astype(int))

In [64]:
# Get total pixel count
res_pred = X_pred["ScreenResolution"].str.extract(r"(\d+)x(\d+)")
X_pred["res_width"] = res_pred[0].astype(int)
X_pred["res_height"] = res_pred[1].astype(int)
X_pred["pixel_count"] = X_pred["res_width"] * X_pred["res_height"]


In [65]:
X_pred = X_pred.drop(columns=["ScreenResolution", "res_width", "res_height"])

In [66]:
# convert memory into total storage
import re

def extract_storage(mem): # create function to extract numeric values and sum
    sizes = re.findall(r"(\d+)(TB|GB)", mem)
    total = 0
    for size, unit in sizes:
        size = int(size)
        if unit == "TB":
            total += size * 1024
        else:
            total += size
    return total

# create new column using the function to get the information
X_pred["storage_gb"] = X_pred["Memory"].apply(extract_storage)

In [67]:
# drop memory column
X_pred = X_pred.drop(columns=["Memory"])

In [68]:
# extract cpu brand
X_pred["cpu_brand"] = X_pred["Cpu"].str.extract(
    r"(Intel|AMD)", expand=False
).str.lower()

In [69]:
X_pred = X_pred.drop(columns=["Cpu"])

In [70]:
X_pred["gpu_brand"] = "other"

In [71]:
X_pred.loc[X_pred["Gpu"].str.contains("intel", case=False, na=False), "gpu_brand"] = "intel"
X_pred.loc[X_pred["Gpu"].str.contains("nvidia", case=False, na=False), "gpu_brand"] = "nvidia"
X_pred.loc[X_pred["Gpu"].str.contains("amd|radeon", case=False, na=False), "gpu_brand"] = "amd"




In [72]:
X_pred = pd.get_dummies(X_pred, columns = ["gpu_brand"], drop_first=True, dtype=int)


In [73]:
X_pred = X_pred.drop(columns=["Gpu"])

In [74]:
X_pred["Weight"] = (X_pred["Weight"].str.replace("kg", "", regex=False).astype(float))

In [75]:
numeric_columns = [
    "Inches",
    "Ram",
    "pixel_count",
    "storage_gb",
    "Weight"
]

In [76]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_pred[numeric_columns] = scaler.fit_transform(
    X_pred[numeric_columns]
)

In [77]:
X_pred.shape

(391, 38)

In [78]:
X_pred.head()

,Inches,Ram,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,...,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand,gpu_brand_intel,gpu_brand_nvidia,gpu_brand_other
laptop_ID,,,,,,,,,,,,,,,,,,,,,
209,0.357415,1.420931,0.502123,0,0,0,0,0,0,0,...,0,0,0,0,-0.110393,-0.255553,intel,0,1,0
1281,0.357415,-0.902069,0.502123,0,0,0,0,0,0,0,...,0,0,0,0,-0.800022,-0.279659,intel,1,0,0
1168,0.357415,-0.902069,-0.250117,0,0,0,0,0,0,0,...,0,0,0,0,-0.800022,0.772988,intel,1,0,0
1231,0.357415,-0.127735,0.187687,0,0,0,1,0,0,0,...,1,0,0,0,-0.110393,-0.769823,intel,1,0,0
1020,-0.786899,-0.902069,-0.174893,0,0,0,0,0,0,1,...,1,0,0,0,-0.110393,-0.769823,intel,1,0,0


In [79]:
X_pred = X_pred.reindex(
    columns=X_train.columns,
    fill_value=0
)



In [80]:
X_pred.head()

,Inches,Ram,Weight,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,...,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,pixel_count,storage_gb,cpu_brand_intel,gpu_brand_intel,gpu_brand_nvidia
laptop_ID,,,,,,,,,,,,,,,,,,,,,
209,0.357415,1.420931,0.502123,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.110393,-0.255553,0,0,1
1281,0.357415,-0.902069,0.502123,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.800022,-0.279659,0,1,0
1168,0.357415,-0.902069,-0.250117,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.800022,0.772988,0,1,0
1231,0.357415,-0.127735,0.187687,0,0,0,1,0,0,0,...,0,1,0,0,0,-0.110393,-0.769823,0,1,0
1020,-0.786899,-0.902069,-0.174893,0,0,0,0,0,0,1,...,0,1,0,0,0,-0.110393,-0.769823,0,1,0


In [126]:
predictions_submit = grid_lgbm.predict(X_pred)
predictions_submit

array([1322.83948249,  418.74115936,  235.88666578, 1029.8922482 ,
        613.92750054,  463.84014494,  759.2962849 , 1192.71261123,
       1071.30473495,  275.33210996, 2547.7819072 , 1320.82888176,
        375.83251471, 1885.16899455,  585.36624313,  520.6310456 ,
       2105.68304844, 1318.70961203, 1841.99073805,  522.59598575,
       1599.20739174,  255.05437982,  537.60367256, 1056.41210659,
        416.16847568,  699.31109911,  502.33263431, 1061.79906876,
       2620.40166299, 1020.60395637, 2433.3263569 ,  339.82749249,
        608.20533863, 2486.63917249, 2068.23619273, 1929.31580256,
        393.90844556, 1386.1084849 ,  790.11076885, 1618.10284114,
        491.61036967, 1322.28820921,  437.76859511, 1293.91753673,
       1719.32667709, 1245.03845691, 1074.4875647 ,  504.93588768,
        763.33397296,  450.57210368, 1734.72255974,  780.71268234,
       1099.39896405,  502.99557732, 1736.58878098, 1687.90071847,
        503.08644185,  790.67805063, 1063.12263543,  655.01355

**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [ ]:
# ¿Qué opináis?
# ¿Sí, no?

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [127]:
sample = pd.read_csv("data/sample_submission.csv")

In [128]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [83]:
sample.shape

(391, 2)

In [129]:
sample["Price_in_euros"] = predictions_submit

In [130]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1322.839482
1,1281,418.741159
2,1168,235.886666
3,1231,1029.892248
4,1020,613.927501


## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [115]:
#¿Cómo creamos la submission?
submission = pd.DataFrame(sample)

In [131]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1309.889662
1,1281,441.392157
2,1168,240.624341
3,1231,1007.120380
4,1020,607.977814


In [87]:
submission.shape

(391, 2)

## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [132]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [133]:
chequeador(submission)

You're ready to submit!
